In [7]:
from google.colab import drive
drive.mount('/content/drive')


MessageError: Error: credential propagation was unsuccessful

In [9]:
pip install pytube

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.0 MB/s eta 0:00:00


In [11]:
pip install opencv-python moviepy


In [16]:
import os
import cv2
import numpy as np

class VideoPreprocessor:
    def __init__(self, input_video_path, output_frame_path, width, height, fps, num_samples):
        self.input_video_path = input_video_path
        self.output_frame_path = output_frame_path
        self.width = width
        self.height = height
        self.fps = fps
        self.num_samples = num_samples

    def preprocess_video(self):
        # Load the video
        video_capture = cv2.VideoCapture(self.input_video_path)

        frame_count = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
        sampled_frames = np.linspace(0, frame_count - 1, self.num_samples, dtype=int)

        frames = []
        current_frame = 0

        # Load YOLO model
        net = cv2.dnn.readNet("/content/./yolov3.weights", "/content/./yolov3.cfg")
        classes = []
        with open("/content/./coco.names", "r") as f:
            classes = [line.strip() for line in f.readlines()]

        # Iterate through the frames and process them
        while True:
            ret, frame = video_capture.read()
            if not ret:
                break

            if current_frame in sampled_frames:
                # Perform object detection
                detected_frame = self.detect_objects(frame)

                # Append to frames list
                frames.append(detected_frame)

            current_frame += 1

        # Release the VideoCapture object
        video_capture.release()

        # Convert frames list to numpy array
        frames_array = np.array(frames)

        # Save frames to numpy array
        np.save(self.output_frame_path, frames_array)

    def detect_objects(self, frame):
        # Load YOLO model
        net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
        classes = []
        with open("coco.names", "r") as f:
            classes = [line.strip() for line in f.readlines()]

        height, width = frame.shape[:2]

        # Preprocess input frame
        blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)

        # Set input to the model
        net.setInput(blob)

        # Perform inference
        outs = net.forward()

        # Compile detection results
        detections = []
        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5:
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)
                    detections.append([class_id, classes[class_id], confidence, (x, y, w, h)])

        # Draw bounding boxes on the frame
        for detection in detections:
            class_id, class_name, confidence, bbox = detection
            x, y, w, h = bbox
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, f"{class_name}: {confidence:.2f}", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        return frame

# Example usage:
if __name__ == "__main__":
    # Example input video paths
    input_video_paths = [
        "/content/./WhatDoesHigh-QualityPreschoolLookLikeNPREd.mp4",
        "/content/./HowGreenRoofsCanHelpCitiesNPR.mp4",
        "/content/./WhyItsUsuallyHotterInACityLetsTalkNPR.mp4"
    ]

    # Example output directory
    output_base_directory = "/content/./output_directory"

    # Parameters for preprocessing
    width = 640
    height = 480
    fps = 30

    # Create output paths for each input video
    for input_path in input_video_paths:
        # Extract the filename from the input video path
        video_name = os.path.splitext(os.path.basename(input_path))[0]

        # Generate the output directory path based on the input video's name
        output_directory = os.path.join(output_base_directory, f"{video_name}_output")

        # Create the output directory if it doesn't exist
        if not os.path.exists(output_directory):
            os.makedirs(output_directory)
            print(f"Directory '{output_directory}' created successfully.")
        else:
            print(f"Directory '{output_directory}' already exists.")

        # Generate the output frame path
        output_frame_path = os.path.join(output_directory, f"{video_name}_frames.npy")

        # Parameters for frame sampling
        num_samples = 100  # adjust the number of samples as needed

        # Perform video preprocessing
        preprocessor = VideoPreprocessor(input_path, output_frame_path, width, height, fps, num_samples)
        preprocessor.preprocess_video()


Directory '/content/./output_directory/WhatDoesHigh-QualityPreschoolLookLikeNPREd_output' already exists.
Directory '/content/./output_directory/HowGreenRoofsCanHelpCitiesNPR_output' already exists.
Directory '/content/./output_directory/WhyItsUsuallyHotterInACityLetsTalkNPR_output' already exists.


In [ ]:
"""
For the code above, if no captions are available for the video, the API will return a message indicating so.
If there are any errors during the download process, the API will provide error messages for troubleshooting.
"""

'\nFor the code above, if no captions are available for the video, the API will return a message indicating so.\nIf there are any errors during the download process, the API will provide error messages for troubleshooting.\n'

In [12]:
import os

# Example output directory
output_directory = "/content/./output_directory"

# Check if the directory exists
if not os.path.exists(output_directory):
    # Create the directory if it doesn't exist
    os.makedirs(output_directory)
    print(f"Directory '{output_directory}' created successfully.")
else:
    print(f"Directory '{output_directory}' already exists.")


Directory '/content/./output_directory' already exists.


In [37]:
import os
import cv2
import numpy as np

class ObjectDetector:
    def __init__(self, config_path, weights_path, names_path, confidence_threshold=0.5):
        self.net = cv2.dnn.readNetFromDarknet(config_path, weights_path)
        self.names = open(names_path).read().strip().split('\n')
        self.confidence_threshold = confidence_threshold

    def detect_objects(self, frame):
        blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)
        self.net.setInput(blob)
        layer_names = self.net.getLayerNames()
        output_layers = [layer_names[i[0] - 1] for i in self.net.getUnconnectedOutLayers()]
        outputs = self.net.forward(output_layers)

        detected_objects = []
        frame_height, frame_width = frame.shape[:2]

        for output in outputs:
            for detection in output:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > self.confidence_threshold:
                    center_x = int(detection[0] * frame_width)
                    center_y = int(detection[1] * frame_height)
                    w = int(detection[2] * frame_width)
                    h = int(detection[3] * frame_height)
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)
                    detected_objects.append((class_id, confidence, (x, y, w, h)))

        return detected_objects

class VideoPreprocessor:
    def __init__(self, input_video_path, output_frame_path, width, height, fps, num_samples):
        self.input_video_path = input_video_path
        self.output_frame_path = output_frame_path
        self.width = width
        self.height = height
        self.fps = fps
        self.num_samples = num_samples

    def preprocess_video(self):
        video_capture = cv2.VideoCapture(self.input_video_path)
        frame_count = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
        sampled_frames = np.linspace(0, frame_count - 1, self.num_samples, dtype=int)
        frames = []
        current_frame = 0

        while True:
            ret, frame = video_capture.read()
            if not ret:
                break

            if current_frame in sampled_frames:
                resized_frame = cv2.resize(frame, (self.width, self.height))
                frames.append(resized_frame)

            current_frame += 1

        video_capture.release()
        frames_array = np.array(frames)
        np.save(self.output_frame_path, frames_array)
        print("Video preprocessing complete.")

# Example usage:
if __name__ == "__main__":
    # Example input video paths
    input_video_paths = [
        "/content/output_directory/HowGreenRoofsCanHelpCitiesNPR_frames.npy",
        "/content/output_directory/WhatDoesHigh-QualityPreschoolLookLikeNPREd_frames.npy",
        "/content/output_directory/WhyItsUsuallyHotterInACityLetsTalkNPR_frames.npy"
    ]

    # Example output directory
    output_base_directory = "/content/output_directory"

    # Parameters for preprocessing
    width = 608
    height = 608
    fps = 30
    confidence_threshold = 0.2

    # Create output paths for each input video
    for input_path in input_video_paths:
        # Extract the filename from the input video path
        video_name = os.path.splitext(os.path.basename(input_path))[0]

        # Generate the output directory path based on the input video's name
        output_directory = os.path.join(output_base_directory, f"{video_name}_output")

        # Create the output directory if it doesn't exist
        if not os.path.exists(output_directory):
            os.makedirs(output_directory)
            print(f"Directory '{output_directory}' created successfully.")
        else:
            print(f"Directory '{output_directory}' already exists.")

        # Generate the output frame path
        output_frame_path = os.path.join(output_directory, f"{video_name}_frames.npy")

        # Parameters for frame sampling
        num_samples = 100  # adjust the number of samples as needed

        # Perform video preprocessing
        preprocessor = VideoPreprocessor(input_path, output_frame_path, width, height, fps, num_samples)
        preprocessor.preprocess_video()

        print(f"Processing video: {input_path}")

        # Load YOLO model
        yolo_config_path = '/content/./yolov3.cfg'
        yolo_weights_path = '/content/./yolov3.weights'
        yolo_names_path = '/content/./coco.names'



Directory '/content/output_directory/HowGreenRoofsCanHelpCitiesNPR_frames_output' already exists.
Video preprocessing complete.
Processing video: /content/output_directory/HowGreenRoofsCanHelpCitiesNPR_frames.npy
Directory '/content/output_directory/WhatDoesHigh-QualityPreschoolLookLikeNPREd_frames_output' already exists.
Video preprocessing complete.
Processing video: /content/output_directory/WhatDoesHigh-QualityPreschoolLookLikeNPREd_frames.npy
Directory '/content/output_directory/WhyItsUsuallyHotterInACityLetsTalkNPR_frames_output' already exists.
Video preprocessing complete.
Processing video: /content/output_directory/WhyItsUsuallyHotterInACityLetsTalkNPR_frames.npy


In [40]:
import os
import cv2
import numpy as np

class ObjectDetector:
    def __init__(self, config_path, weights_path, names_path, confidence_threshold=0.5):
        self.net = cv2.dnn.readNetFromDarknet(config_path, weights_path)
        self.names = open(names_path).read().strip().split('\n')
        self.confidence_threshold = confidence_threshold

    def detect_objects(self, frame):
        blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)
        self.net.setInput(blob)
        layer_names = self.net.getLayerNames()
        output_layers = [layer_names[i[0] - 1] for i in self.net.getUnconnectedOutLayers()]
        outputs = self.net.forward(output_layers)

        detected_objects = []
        frame_height, frame_width = frame.shape[:2]

        for output in outputs:
            for detection in output:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > self.confidence_threshold:
                    center_x = int(detection[0] * frame_width)
                    center_y = int(detection[1] * frame_height)
                    w = int(detection[2] * frame_width)
                    h = int(detection[3] * frame_height)
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)
                    detected_objects.append((class_id, confidence, (x, y, w, h)))

        return detected_objects

class VideoPreprocessor:
    def __init__(self, input_video_path, output_frame_path, width, height, fps, num_samples):
        self.input_video_path = input_video_path
        self.output_frame_path = output_frame_path
        self.width = width
        self.height = height
        self.fps = fps
        self.num_samples = num_samples

    def preprocess_video(self):
        video_capture = cv2.VideoCapture(self.input_video_path)
        frame_count = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
        sampled_frames = np.linspace(0, frame_count - 1, self.num_samples, dtype=int)
        frames = []
        current_frame = 0

        while True:
            ret, frame = video_capture.read()
            if not ret:
                break

            if current_frame in sampled_frames:
                resized_frame = cv2.resize(frame, (self.width, self.height))
                frames.append(resized_frame)

            current_frame += 1

        video_capture.release()
        frames_array = np.array(frames)
        np.save(self.output_frame_path, frames_array)
        print("Video preprocessing complete.")

# Example usage:
if __name__ == "__main__":
    # Example input video paths
    input_video_paths = [
        "/content/output_directory/HowGreenRoofsCanHelpCitiesNPR_frames.npy",
        "/content/output_directory/WhatDoesHigh-QualityPreschoolLookLikeNPREd_frames.npy",
        "/content/output_directory/WhyItsUsuallyHotterInACityLetsTalkNPR_frames.npy"
    ]

    # Example output directory
    output_base_directory = "/content/./output_directory"

    # Parameters for preprocessing
    width = 608
    height = 608
    fps = 30
    confidence_threshold = 0.2

    # Create output paths for each input video
    for input_path in input_video_paths:
        # Extract the filename from the input video path
        video_name = os.path.splitext(os.path.basename(input_path))[0]

        # Generate the output directory path based on the input video's name
        output_directory = os.path.join(output_base_directory, f"{video_name}_output")

        # Generate the output frame path
        output_frame_path = os.path.join(output_directory, f"{video_name}_frames.npy")

        # Parameters for frame sampling
        num_samples = 100  # adjust the number of samples as needed

        # Perform video preprocessing
        preprocessor = VideoPreprocessor(input_path, output_frame_path, width, height, fps, num_samples)
        preprocessor.preprocess_video()

        print(f"Processing video: {input_path}")

        # Load YOLO model
        yolo_config_path = '/content/./yolov3.cfg'
        yolo_weights_path = '/content/./yolov3.weights'
        yolo_names_path = '/content/./coco.names'
        detector = ObjectDetector(yolo_config_path, yolo_weights_path, yolo_names_path, confidence_threshold)

       # Load preprocessed frames
        # Load preprocessed frames
        frames_array = np.load(output_frame_path)

        # Perform object detection on each frame and visualize the results
        for i, frame in enumerate(frames_array):
            detected_objects = detector.detect_objects(frame)
            for obj in detected_objects:
                class_id, confidence, bbox = obj
                class_name = detector.names[class_id]
                x, y, w, h = bbox
                # Draw bounding box
                color = (0, 255, 0)  # Green color
                cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                # Put text indicating the class name and confidence
                text = f"{class_name}: {confidence:.2f}"
                cv2.putText(frame, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            # Display the annotated frame
            cv2.imshow("Frame", frame)
            cv2.waitKey(0)  # Press any key to proceed to the next frame

        # Close the window after processing all frames
        cv2.destroyAllWindows()



    print("Object detection complete.")


Video preprocessing complete.
Processing video: /content/output_directory/HowGreenRoofsCanHelpCitiesNPR_frames.npy
Video preprocessing complete.
Processing video: /content/output_directory/WhatDoesHigh-QualityPreschoolLookLikeNPREd_frames.npy
Video preprocessing complete.
Processing video: /content/output_directory/WhyItsUsuallyHotterInACityLetsTalkNPR_frames.npy
Object detection complete.
